In [3]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import math

In [ ]:

import gurobipy as gp
from gurobipy import GRB
import math

# City coordinates (Euclidean). Keys must match names used in sets below.
coords = {
    "Warsaw": (100, 65),
    "Skierniewice": (92, 62),
    "Łódź": (85, 58),
    "Konin": (68, 64),
    "Sieradz": (72, 55),
    "Piotrków": (89, 51),
    "Radom": (102, 50),
    "Kielce": (98, 42),
    "Czestochowa": (79, 40),
    "Bytom": (76, 31),
    "Gliwice": (73, 30),
    "Sosnowiec": (80, 29),
    "Kraków": (89, 25),
    "Bielsko": (79, 20),
    "Katowice": (78, 28),
    "Jastrzębie-Zdrój": (74, 23),
    "Wodzisław Śląski": (71, 23),
    "Poznań": (54, 68),
    "Zielona Góra": (39, 60),
    "Leszno": (51, 58),
    "Kalisz": (65, 58),
    "Legnica": (45, 47),
    "Wrocław": (54, 45),
    "Jelenia Góra": (40, 42),
    "Wałbrzych": (46, 39),
    "Opole": (67, 37)
}

# Areas (partition of N)
N1 = {"Warsaw", "Skierniewice", "Łódź", "Konin", "Sieradz", "Piotrków", "Radom"}
N2 = {"Kielce", "Czestochowa", "Bytom", "Gliwice", "Sosnowiec",
      "Kraków", "Bielsko", "Katowice", "Jastrzębie-Zdrój", "Wodzisław Śląski"}
N3 = {"Poznań", "Zielona Góra", "Leszno", "Kalisz", "Legnica",
      "Wrocław", "Jelenia Góra", "Wałbrzych", "Opole"}

# Policy sets
K1 = {"Konin", "Kalisz"} & set(coords)           # at least one
K2 = {"Bytom", "Sosnowiec", "Katowice"} & set(coords)  # at most one
K3 = {"Wodzisław Śląski", "Jastrzębie-Zdrój"} & set(coords)  # at most one
K4 = {"Poznań", "Zielona Góra", "Leszno"} & set(coords)      # at least one

# Mandatory edges (directed) — adjust direction if needed
MANDATORY_EDGES = [("Radom", "Kielce"), ("Gliwice", "Opole")]

# Sanity: build N and verify disjoint + cover
N = set(coords)
assert N1 | N2 | N3 <= N, "Area sets must be subset of N"
assert (N1 & N2) == set() and (N1 & N3) == set() and (N2 & N3) == set(), "Areas must be disjoint"


def euclid(a, b):
    ax, ay = coords[a]; bx, by = coords[b]
    return math.hypot(ax - bx, ay - by)

# Precompute distances (no self-arcs)
d = {(i, j): euclid(i, j) for i in N for j in N if i != j}

m = gp.Model("Telecom_Path_SW_Poland")

# Big-M for MTZ ordering (safe upper bound: |N|)
M = len(N)


# x[i,j] = 1 if path uses directed arc i->j
x = m.addVars(d.keys(), vtype=GRB.BINARY, name="x")
# y[i] = 1 if city i is included
y = m.addVars(N, vtype=GRB.BINARY, name="y")
# u[i] = position index in the path (continuous)
u = m.addVars(N, lb=0.0, ub=M, vtype=GRB.CONTINUOUS, name="u")


m.setObjective(gp.quicksum(d[i, j] * x[i, j] for (i, j) in d), GRB.MINIMIZE)



# 1) Start/End degree constraints
# Start: exactly one outgoing from Warsaw; no incoming to Warsaw
m.addConstr(gp.quicksum(x["Warsaw", j] for j in N if j != "Warsaw") == 1, "start_out")
m.addConstr(gp.quicksum(x[i, "Warsaw"] for i in N if i != "Warsaw") == 0, "start_in")

# End: exactly one incoming to Jelenia Góra; no outgoing from Jelenia Góra
m.addConstr(gp.quicksum(x[i, "Jelenia Góra"] for i in N if i != "Jelenia Góra") == 1, "end_in")
m.addConstr(gp.quicksum(x["Jelenia Góra", j] for j in N if j != "Jelenia Góra") == 0, "end_out")

# Flow conservation for intermediate nodes (equal to y_j)
for j in N - {"Warsaw", "Jelenia Góra"}:
    m.addConstr(gp.quicksum(x[i, j] for i in N if i != j) == y[j], f"in_{j}")
    m.addConstr(gp.quicksum(x[j, k] for k in N if k != j) == y[j], f"out_{j}")

# City inclusion for endpoints
m.addConstr(y["Warsaw"] == 1, "y_warsaw")
m.addConstr(y["Jelenia Góra"] == 1, "y_jg")

# 2) Mandatory cities (as per statement)
for city in ["Radom", "Kielce", "Gliwice", "Opole"]:
    if city in N:
        m.addConstr(y[city] == 1, f"mandatory_{city}")

# 3) Area coverage counts
m.addConstr(gp.quicksum(y[i] for i in N1) == 4, "area1_count")
m.addConstr(gp.quicksum(y[i] for i in N2) == 5, "area2_count")
m.addConstr(gp.quicksum(y[i] for i in N3) == 5, "area3_count")

# 4) Policy constraints
if K1: m.addConstr(gp.quicksum(y[i] for i in K1) >= 1, "policy_K1")
if K2: m.addConstr(gp.quicksum(y[i] for i in K2) <= 1, "policy_K2")
if K3: m.addConstr(gp.quicksum(y[i] for i in K3) <= 1, "policy_K3")
if K4: m.addConstr(gp.quicksum(y[i] for i in K4) >= 1, "policy_K4")

# 5) No bidirectional connections
for i in N:
    for j in N:
        if i < j:  # avoid duplicates
            if (i, j) in x and (j, i) in x:
                m.addConstr(x[i, j] + x[j, i] <= 1, f"nobidir_{i}_{j}")

# 6) Edge-activation implies node-activation
for (i, j) in d:
    m.addConstr(x[i, j] <= y[i], f"x_le_yi_{i}_{j}")
    m.addConstr(x[i, j] <= y[j], f"x_le_yj_{i}_{j}")

# 7) Mandatory existing infrastructure edges fixed ON
for (a, b) in MANDATORY_EDGES:
    if (a in N) and (b in N) and (a, b) in x:
        m.addConstr(x[a, b] == 1, f"mandatory_edge_{a}_{b}")
        # These imply y[a]=y[b]=1 already via edge->node constraints

# 8) MTZ subtour elimination (path version)
# u[Warsaw] = 1, others between 0 and M*y[i]
m.addConstr(u["Warsaw"] == 1, "u_start")
for i in N - {"Warsaw"}:
    # If y[i] = 0 -> allow u[i] = 0; if y[i] = 1 -> 1..M
    m.addConstr(u[i] >= y[i], f"u_lb_{i}")       # lower bound activates with y
    m.addConstr(u[i] <= M * y[i], f"u_ub_{i}")  # upper bound deactivates with y

# MTZ core: for all i != j, enforce order when x[i,j]=1
for i in N:
    for j in N:
        if i != j and (i, j) in x:
            # u_i - u_j + M*x_ij <= M - 1
            # This naturally allows when x_ij=0; when x_ij=1 => u_i + 1 <= u_j
            m.addConstr(u[i] - u[j] + M * x[i, j] <= M - 1, f"mtz_{i}_{j}")

# ---------- SOLVE ----------
m.optimize()

# ---------- EXTRACT SOLUTION ----------
if m.status == GRB.OPTIMAL:
    print(f"Objective (total length): {m.objVal:.3f}")
    selected_nodes = [i for i in N if y[i].X > 0.5]
    print("Selected cities:", selected_nodes)

    # Build successor map and reconstruct path
    succ = {i: None for i in N}
    for (i, j), var in x.items():
        if var.X > 0.5:
            succ[i] = j

    path = ["Warsaw"]
    while succ[path[-1]] is not None:
        path.append(succ[path[-1]])
        if len(path) > len(N):  # safety
            break
    print("Path:", " -> ".join(path))
else:
    print("No optimal solution found. Status:", m.status)


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 23.6.0 23H626)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2393 rows, 702 columns and 6693 nonzeros
Model fingerprint: 0x85b85372
Variable types: 26 continuous, 676 integer (676 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [2e+00, 6e+01]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 2e+01]
Presolve removed 1637 rows and 161 columns
Presolve time: 0.02s
Presolved: 756 rows, 541 columns, 3135 nonzeros
Variable types: 22 continuous, 519 integer (519 binary)
Found heuristic solution: objective 349.2201849

Root relaxation: objective 1.339359e+02, 127 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  133.93595    0   34  349.22018  133.9